In [3]:
import mlflow
import mlflow.sklearn
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression

data = pd.DataFrame({
    'square_footage': [1500, 2000, 2500, 1800, 3000],
    'price': [300000, 400000, 500000, 350000, 600000],
    'neighborhood': ['A', 'B', 'A', 'B', 'A']
})



# Set the MLflow experiment
mlflow.set_experiment("House Pricing Prediction")

# Custom transformer for cost_per_square_foot
class CostPerSquareFootTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        #self.y_ = y  # Save y for transformation
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X['cost_per_square_foot'] = X['square_footage'] / y
        return X

# Preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['square_footage', 'cost_per_square_foot']),
        ('cat', OneHotEncoder(), ['neighborhood'])
    ],
    remainder='drop'
)

# Final pipeline
pipeline = Pipeline(steps=[
    ('feature_engineering', CostPerSquareFootTransformer()),  # Custom feature engineering
    ('preprocessor', preprocessor),  # Preprocessing
    ('model', LinearRegression())  # Model
])

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    data[['square_footage', 'neighborhood']],
    data['price'],
    test_size=0.2,
    random_state=42
)

# Train the model with MLflow tracking
with mlflow.start_run():
    # Log parameters (pipeline components)
    mlflow.log_param("model", "LinearRegression")
    mlflow.log_param("preprocessor", "StandardScaler and OneHotEncoder")
    mlflow.log_param("custom_transformer", "CostPerSquareFootTransformer")

    # Fit the pipeline
    pipeline.fit(X_train, y_train)

    # Make predictions
    y_pred = pipeline.predict(X_test)

    # Calculate metrics
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)

    # Log metrics
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

    # Log the entire pipeline as an artifact
    mlflow.sklearn.log_model(pipeline, "pipeline_model")

    # End the run
    print(f"Run complete: RMSE={rmse}, R2={r2}")

# Load the logged model (for testing or deployment)
logged_model = "runs:/<run_id>/pipeline_model"
loaded_pipeline = mlflow.sklearn.load_model(logged_model)

# Make predictions with the loaded model
y_loaded_pred = loaded_pipeline.predict(X_test)


2025/01/23 20:43:38 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



TypeError: unsupported operand type(s) for /: 'int' and 'NoneType'